## This is for the exam project


In [2]:
##Imports
from time import sleep
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup as soup
import bs4
import requests
import re

In [32]:
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Cellar/tesseract/4.1.1/bin/tesseract'

img = cv2.imread("./images/numberplate3.png")
text = pytesseract.image_to_string(img)
print(text)


BW 80 777

 



In [10]:
print("Starting up..")
profile = webdriver.FirefoxProfile()
profile.set_preference("general.useragent.override", "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:81.0) Gecko/20100101 Firefox/81.0")
options = Options()
options.headless = True

browser = webdriver.Firefox(options=options)

browser.get("https://motorregister.skat.dk/dmr-kerne/koeretoejdetaljer/visKoeretoej?execution=e2s1")

print("Got Browser..")

browser.implicitly_wait(3)

#cookie_button = browser.find_element_by_css_selector('div.coi-banner__page-footer:nth-child(2) > button:nth-child(2)')
#cookie_button.click()
#print("Cookie Button Clicked..")
#sleep(3) 

search_field = browser.find_element_by_id('soegeord')

search_field.send_keys("BW80777")
search_field.submit()
print("Search Entered..")
sleep(3)

print()

page_source = browser.page_source
page_soup = soup(page_source,'html.parser')
res = page_soup.select_one("div > div.cols2 > div.colNo1.unit > div > div:nth-child(2) > span.value").text
print("Søgte bil: ", res)
browser.quit()

Starting up..
Got Browser..
Search Entered..



AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
browser.quit()